# Import client

In [9]:
import pymongo
from dbclient import DBClient

In [10]:
db_client = DBClient()
db_client.check_connection()
db_client.collections.keys()

Pinged your deployment. You successfully connected to MongoDB!


dict_keys(['members', 'athletes', 'coaches', 'activities', 'session_types', 'access_types'])

# Lookup tables

In [12]:
db_client.clear_all("access_types")

access_types_lookup = [
    {"accessId": None, "description": "No access to any application data"},
    {"accessId": 0, "description": "Full access to the application"},
    {"accessId": 1, "description": "Access to the personal data only"},
    {"accessId": 2, "description": "Access to the REPS data only"},
]

db_client.collections["access_types"].insert_many(access_types_lookup)

db_client.collections["access_types"].create_index([("accessId", pymongo.ASCENDING)], unique=True)

True
All entries successfully cleared from access_types table.


'accessId_1'

In [12]:
member_data = [
    {"memberId": 1, "firstName": "Ben", "lastName": "Hoskings"}
]